In [1]:
import pandas as pd
import numpy as np


### Importing and cleaning the historical file

In [2]:
#Import glob loop through 20 years of storm data
import glob

path = "Resources" 
all_files = glob.glob(path + "/*.csv")
li=[]

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Create a dataframe using the imported data
storm_data = pd.concat(li, axis=0, ignore_index=True)

storm_data.head()

C:\Users\meron\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (29,34,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,199901,14,0,199901,15,0,1501526,5683694,MASSACHUSETTS,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A strong high pressure system centered over so...,NaN,PDC
1,199901,2,700,199901,2,2359,1500622,5683477,MICHIGAN,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blizzard conditions developed across lower Mic...,NaN,PDC
2,199901,13,130,199901,13,1400,1500140,5683047,OHIO,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Freezing rain and sleet changed to snow as a s...,NaN,PDC
3,199908,1,1,199908,31,2359,2409403,5712941,MISSOURI,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"After an abnormally wet June, the rest of the ...",NaN,PDC
4,199901,2,2300,199901,3,1900,1502756,5681541,NEW YORK,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A deep area of low pressure moved from the Mis...,NaN,PDC


In [3]:
#Parse year and month description
storm_data['BEGIN_YEARMONTH']=storm_data['BEGIN_YEARMONTH'].astype(str)
storm_data['BEGIN_YEAR']=storm_data['BEGIN_YEARMONTH'].str[0:4]
storm_data['BEGIN_MONTH']=storm_data['BEGIN_YEARMONTH'].str[4:6]

#Move the new columns to the first and second column
cols = list(storm_data.columns)
cols = [cols[-1]] + cols[:-1]
storm_data = storm_data[cols]

cols = list(storm_data.columns)
cols = [cols[-1]] + cols[:-1]
storm_data = storm_data[cols]


In [4]:
storm_data.head()

,BEGIN_YEAR,BEGIN_MONTH,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,1999,01,199901,14,0,199901,15,0,1501526,5683694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A strong high pressure system centered over so...,NaN,PDC
1,1999,01,199901,2,700,199901,2,2359,1500622,5683477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blizzard conditions developed across lower Mic...,NaN,PDC
2,1999,01,199901,13,130,199901,13,1400,1500140,5683047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Freezing rain and sleet changed to snow as a s...,NaN,PDC
3,1999,08,199908,1,1,199908,31,2359,2409403,5712941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"After an abnormally wet June, the rest of the ...",NaN,PDC
4,1999,01,199901,2,2300,199901,3,1900,1502756,5681541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A deep area of low pressure moved from the Mis...,NaN,PDC


In [5]:
#Retrive the total for damage and death
storm_data[["DEATHS_DIRECT",'DEATHS_INDIRECT','INJURIES_DIRECT','INJURIES_INDIRECT',"DAMAGE_PROPERTY","DAMAGE_CROPS"]].fillna(0, inplace=True)
storm_data["DEATH_TOLL"]=storm_data['DEATHS_DIRECT']+storm_data['DEATHS_INDIRECT']
storm_data['TOTAL_INJURIES']=storm_data['INJURIES_DIRECT']+storm_data['INJURIES_INDIRECT']

C:\Users\meron\Anaconda3\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [6]:
#change property and crop damage to numbers and add 1000
storm_data['DAMAGE_PROPERTY']=storm_data['DAMAGE_PROPERTY'].str.replace("K",'000')
storm_data['DAMAGE_PROPERTY']=storm_data['DAMAGE_PROPERTY'].str.replace('M','000000')
storm_data['DAMAGE_PROPERTY']=storm_data['DAMAGE_PROPERTY'].str.replace('B','000000000')
storm_data["DAMAGE_CROPS"]=storm_data['DAMAGE_CROPS'].str.replace('M','000000')
storm_data["DAMAGE_CROPS"]=storm_data['DAMAGE_CROPS'].str.replace('K','000')
storm_data["DAMAGE_CROPS"]=storm_data['DAMAGE_CROPS'].str.replace('B','000000000')


In [7]:
storm_data["DAMAGE_PROPERTY"]=(storm_data['DAMAGE_PROPERTY'].apply(pd.to_numeric))
storm_data["DAMAGE_CROPS"]=(storm_data['DAMAGE_CROPS'].apply(pd.to_numeric))

In [8]:
storm_data.head()

,BEGIN_YEAR,BEGIN_MONTH,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,...,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE,DEATH_TOLL,TOTAL_INJURIES
0,1999,01,199901,14,0,199901,15,0,1501526,5683694,...,NaN,NaN,NaN,NaN,NaN,A strong high pressure system centered over so...,NaN,PDC,0,0
1,1999,01,199901,2,700,199901,2,2359,1500622,5683477,...,NaN,NaN,NaN,NaN,NaN,Blizzard conditions developed across lower Mic...,NaN,PDC,0,0
2,1999,01,199901,13,130,199901,13,1400,1500140,5683047,...,NaN,NaN,NaN,NaN,NaN,Freezing rain and sleet changed to snow as a s...,NaN,PDC,0,0
3,1999,08,199908,1,1,199908,31,2359,2409403,5712941,...,NaN,NaN,NaN,NaN,NaN,"After an abnormally wet June, the rest of the ...",NaN,PDC,0,0
4,1999,01,199901,2,2300,199901,3,1900,1502756,5681541,...,NaN,NaN,NaN,NaN,NaN,A deep area of low pressure moved from the Mis...,NaN,PDC,0,0


In [10]:
storm_data.to_csv("Output/Storm Data.csv")